# Capstone Project - The Battle of Neighborhoods

This is my contribution to the final Peer-Reviewed Assignment for the Capstone Project for Applied Data Science Specialization from IBM on Coursera.

## Section 1: Introduction

In this section, I will define the idea of my choosing, where I use Foursquare's places API to solve an assumed business opportunity.

### Background

Shopping hours in Germany are very restrictive. Most shops are closed on Sundays, and few shops remain open after 8:00 PM on weekdays. Finding open shops at night is difficult, especially in the winter session. There are many unhappy customers and opportunities for businesses to open and serve in metropolitan cities where people from different cultures live.

This project will map the shops with extended working times and analyze how many households they serve and which neighborhoods may need a shop with more flexible shopping hours. This analysis will focus on Frankfurt am Main, which has 763,380 inhabitants as of December 31, 2019, and one of the global hubs for commerce, culture, education, tourism, and transportation.

The project will help entrepreneurs and shop owners realize the untapped opportunity. It is also assisting consumers in finding neighborhoods with stores that open at the typical closing time.

### Data

Given the project size and simplicity, I will use a few data sources in a narrow context. In this project, I will use the Foursquare API as a primary data source. Foursquare offers various end-points that can be utilized to execute the project's idea.

I will use the [Venue Categories](https://developer.foursquare.com/docs/api-reference/venues/categories/) end-point to get the list of the available categories, focus on the categories related to retailer shops and supermarkets. Then use the [Venue Search](https://developer.foursquare.com/docs/api-reference/venues/search/) to get a list of the venues using the defined categories and a limited radius. After that, I will use the [Venue Hours](https://developer.foursquare.com/docs/api-reference/venues/hours/) premium end-point to get the information about the opening hours.